# serverless endpoint

https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html

In [1]:
# !pip3 install sagemaker
#!pip3 install --upgrade botocore

In [2]:
import json
from abcli import fullname
from abcli import string
from abcli.plugins import cache
from roofAI import NAME, VERSION
from roofAI.inference.classes import InferenceClient, InferenceObject
from roofAI.logger import logger

logger.info(f"{NAME}-{VERSION}-{fullname()}, built on {string.pretty_date()}")

Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


roofAI-3.271.1-abcli-7.2564.1-unknown, built on 10 December 2023, 18:41:31


In [3]:
model_name = cache.read("roofAI_semseg_model_AIRS_o2")

In [4]:
inference_client = InferenceClient(verbose=True)

Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name shamim to get Role path.
sagemaker_role: defaulting to local.
Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.
InferenceClient created.


In [5]:
assert inference_client.create_model(name=model_name)

create_model(model-2023-12-03-11-24-39-75649)...
list(InferenceObject.MODEL,model-2023-12-03-11-24-39-75649): {'Models': [{'ModelName': 'model-2023-12-03-11-24-39-75649', 'ModelArn': 'arn:aws:sagemaker:ca-central-1:120429650996:model/model-2023-12-03-11-24-39-75649', 'CreationTime': datetime.datetime(2023, 12, 10, 18, 41, 0, 379000, tzinfo=tzlocal())}], 'ResponseMetadata': {'RequestId': '1c232f03-d444-496b-9f86-7d611036373e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1c232f03-d444-496b-9f86-7d611036373e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '187', 'date': 'Mon, 11 Dec 2023 02:41:33 GMT'}, 'RetryAttempts': 0}}
delete(InferenceObject.MODEL,model-2023-12-03-11-24-39-75649): {'ResponseMetadata': {'RequestId': '2297301c-4ea3-4bb6-b905-e9dffa2bcea5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2297301c-4ea3-4bb6-b905-e9dffa2bcea5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Mon, 11 Dec 2023 02:41

In [6]:
config_name =  "config-{}-{}".format(model_name, string.random_(8))
assert inference_client.create_endpoint_config(name=config_name, model_name=model_name)

create_endpoint_config(config-model-2023-12-03-11-24-39-75649-Wnmubt9u:model-2023-12-03-11-24-39-75649)...
list(InferenceObject.ENDPOINT_CONFIG,config-model-2023-12-03-11-24-39-75649-Wnmubt9u): {'EndpointConfigs': [], 'ResponseMetadata': {'RequestId': 'cd9ba9de-54b6-41af-8273-1fc4b7e30e45', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cd9ba9de-54b6-41af-8273-1fc4b7e30e45', 'content-type': 'application/x-amz-json-1.1', 'content-length': '22', 'date': 'Mon, 11 Dec 2023 02:41:34 GMT'}, 'RetryAttempts': 0}}
{'EndpointConfigArn': 'arn:aws:sagemaker:ca-central-1:120429650996:endpoint-config/config-model-2023-12-03-11-24-39-75649-wnmubt9u', 'ResponseMetadata': {'RequestId': '22890641-2040-497a-8292-2db8dc1b1221', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '22890641-2040-497a-8292-2db8dc1b1221', 'content-type': 'application/x-amz-json-1.1', 'content-length': '131', 'date': 'Mon, 11 Dec 2023 02:41:34 GMT'}, 'RetryAttempts': 0}}
list(InferenceObject.ENDPOINT_CONFIG

In [7]:
endpoint_name = "endpoint-{}-{}".format(model_name, string.random_(8))
assert inference_client.create_endpoint(name=endpoint_name, config_name=config_name)

create_endpoint(endpoint-model-2023-12-03-11-24-39-75649-AonRGlLb:config-model-2023-12-03-11-24-39-75649-Wnmubt9u)...
list(InferenceObject.ENDPOINT,endpoint-model-2023-12-03-11-24-39-75649-AonRGlLb): {'Endpoints': [], 'NextToken': 'cIws2QhTXUIa8bi8XNKmlzPAFyLsu1lXMD9a1y2x7vEtL8zPsPwGycEctt6JW+eTQvknOfRTFGHpHd9NH3CinlDyMohuxRq4WqvqjpyBjRrmYNDuXZUeTwx2efZ9V0F2iKTxVWlN1UJSXXqtnJpaYVf4Dg74gIWmpVtEFLpGmR/xlNm2TNoAQMhsZa94l32/zbw+MOe0RK+Vnc6VogrEDh5c/VidIQIkHJa6+wMXSFHEkY5jwzbpXJFgDIO453IdSY/xEQSeoYHHiwaYtVwdes04FZ9y/P7vJy+GOwptzC77zTdOttim4iavDE5+TgF9qXTh3d0yjmV3x66Voqe7N9RT7IYeQawGf4IaVKcsyTc7Lm17NR/twSiZIQcmrTH1lmZdSNzpbzdTJZdzyhKNNogMo3GxLP03Rzz6SGnpVXCF3l74sgqmRw89mVmIpZijdWP4+ZG16lpYK8BCohXQ7gmHdz5vLhakSqxrCWvFrGvgVJrsc6LNLZ4aDOvmP15pwpUTS9CgARneVSCCebO7xOEM3baWkd1FYJ4e+ImM+v9hNihqty5Kdy8MBggfEHXAwUOrDuUDZ+LSIA2jT/mqqF+nI4byyvMoBQrPQ55Y/HHPV8DLWg9AfOcp/HRlDNbSh+TSoH7aeWE=', 'ResponseMetadata': {'RequestId': '5b8a6f64-ae78-4a2d-b848-3e59a4e2eb87', 'HTTPStatusCode': 200, 'HTTPHeaders': {

In [8]:
# END